In [1]:
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import xgboost as xgb
import numpy as np
import pandas as pd 
import random
import pickle
from mlforecast import MLForecast
import plotly.express as px
from argparse import Namespace
import warnings
import torch
import os
import holidays
from pathlib import Path
from tqdm import tqdm
from models import TimesNet, iTransformer
from data_factory import data_provider, holiday_dates
import pickle

result_path = 'results/'
log_path = 'logs/'
root_path = os.getcwd()
data_path = 'data/'


random.seed(42)
np.random.seed(42)

warnings.filterwarnings('ignore')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)

device: cuda:0


# NeuralProphet

In [3]:
dt_name = 'wind_plant'
with open(result_path + f'neural_prophet_{dt_name}_results.pkl', 'rb') as f:
    np_model_results =  pickle.load(f)

In [4]:
for i, model_record in enumerate(np_model_results):
    
    if model_record['is_final']:
        print("Test MAE:", model_record['full_val_mae'])
        future_preds = model_record['future_preds']
        n_lags, n_feature_lags =  model_record['specs']
        fig = px.line(future_preds)
        fig.update_layout(title= f'{dt_name} Forecasting by NeuralProphet with n_lags={n_lags} and n_feature_lags={n_feature_lags}')
        fig.show()

Test MAE: 8.518658507377822


In [5]:
dt_name = 'electricity_demand'
with open(result_path + f'neural_prophet_{dt_name}_results.pkl', 'rb') as f:
    np_model_results =  pickle.load(f)

In [6]:
for i, model_record in enumerate(np_model_results):
    
    if model_record['is_final']:
        print("Test MAE:", model_record['full_val_mae'])
        future_preds = model_record['future_preds']
        n_lags, n_feature_lags =  model_record['specs']
        fig = px.line(future_preds)
        fig.update_layout(title= f'{dt_name} Forecasting by NeuralProphet with n_lags={n_lags} and n_feature_lags={n_feature_lags}')
        fig.show()
        future_df = pd.read_csv(data_path + f'{dt_name}/test.csv', index_col=0)
        future_df['demand'] = future_preds
        future_df.to_csv(result_path + f"pred_{dt_name}_final_neuralprophet_lag{n_lags}.csv")

Test MAE: 1572.3409573709696


In [31]:
from json import load
from sys import argv

def loc(nb):
    cells = load(open(nb, encoding='utf-8'))['cells']
    return sum(len(c['source']) for c in cells if c['cell_type'] == 'code')

def run(ipynb_files):
    return sum(loc(nb) for nb in ipynb_files)

print(run(["Model Evaluation.ipynb", "Electricity Demand Analysis.ipynb", "Wind Power Plant Production Analysis.ipynb", "Electricity Demand Forecasting - NeuralProphet.ipynb", "Wind Plant Production Forecasting - NeuralProphet.ipynb"]))

1067


# LightGBM

In [7]:
from gradboost_tuning import gradboost_tuning_main

dt_name = 'wind_plant'
target = 'prod'
model_type = 'lightgbm'
n_lags = 1
gradboost_tuning_main(dt_name, target, model_type, n_lags, None, None, None, True)

Test MAE: 6.795902434560355


In [8]:
gradboost_tuning_main(dt_name, target, model_type, n_lags, None, None, None, False)

>>> Saved: results/pred_wind_plant_final_lightgbm_lag1.csv


In [ ]:
from gradboost_tuning import gradboost_tuning_main

dt_name = 'electricity_demand'
target = 'demand'
model_type = 'lightgbm'
n_lags = 168
gradboost_tuning_main(dt_name, target, model_type, n_lags, None, None, None, True)

<div class="alert alert-block alert-info">
⚠️ LightGBM models fail with the recursive electricity demand forecasting!!! So, it is not usable and should be probed later.
</div>


# iTransformer

Parsing tuning records from wandb.ai

In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("furkancanturk/electricity_demand_forecasting")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    result = run.summary._json_dict
    summary_list.append(result)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df = pd.concat((runs_df['name'], pd.DataFrame(list(runs_df['config'].values)), pd.DataFrame(list(runs_df['summary'].values))), axis=1)


In [32]:
import pandas as pd 
import wandb
api = wandb.Api()

dt_name = 'wind_plant'
# Project is specified by <entity/project-name>
runs = api.runs(f"furkancanturk/{dt_name}_forecasting")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    result = run.summary._json_dict
    summary_list.append(result)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df = pd.concat((runs_df['name'], pd.DataFrame(list(runs_df['config'].values)), pd.DataFrame(list(runs_df['summary'].values))), axis=1)


Selecting the best 1h and 24h models

In [33]:
models_1h = runs_df.groupby('pred_len').get_group(1).reset_index(drop=True)
models_24h = runs_df.groupby('pred_len').get_group(24).reset_index(drop=True)

best_model_1h_record = models_1h.iloc[models_1h['MAE_val'].idxmin()]
best_model_24h_record = models_24h.iloc[models_24h['MAE_val'].idxmin()]

print(best_model_1h_record['MAE_val'])

0.14999601244926453


Loading the checkpoints of the best models

In [34]:
model_name = best_model_1h_record['name']
setting = dt_name + "/" + model_name
model_1h_config = args = Namespace(**best_model_1h_record.to_dict())
model_1h = iTransformer.Model(model_1h_config)
file_name = './checkpoints/' + setting + "/" + 'checkpoint.pth'
model_1h.load_state_dict(torch.load(file_name))
model_1h = model_1h.to(device)

model_name = best_model_24h_record['name']
setting = dt_name + "/" + model_name
model_24h_config = args= Namespace(**best_model_24h_record.to_dict())
model_24h = iTransformer.Model(model_24h_config)
file_name = './checkpoints/' + setting + "/" + 'checkpoint.pth'
model_24h.load_state_dict(torch.load(file_name))
model_24h = model_24h.to(device)

models = [(model_1h_config, model_1h), (model_24h_config, model_24h)]

### Testing the Best Models

In [11]:
def pytorch_test_forecasting(model, args, test_data, test_loader):
   
    preds = []
    trues = []

    model.eval()
    with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):

            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)

            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)

            # decoder input
            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)

            # encoder - decoder
            if args.use_amp:
                with torch.cuda.amp.autocast():
                    if args.output_attention:
                        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
            else:
                if args.output_attention:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]

                else:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, :]
            batch_y = batch_y[:, -args.pred_len:, :].to(device)
            outputs = outputs.detach().cpu().numpy()
            batch_y = batch_y.detach().cpu().numpy()
            
            if test_data.scale and args.inverse:
                shape = outputs.shape
                outputs = test_data.inverse_transform(outputs.squeeze(0)).reshape(shape)
                batch_y = test_data.inverse_transform(batch_y.squeeze(0)).reshape(shape)
    
            outputs = outputs[:, :, f_dim:]
            batch_y = batch_y[:, :, f_dim:]

            preds.append(outputs)
            trues.append(batch_y)

    preds = np.array(preds)
    trues = np.array(trues)

    print('test shape:', preds.shape, trues.shape)
    preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1]).squeeze()
    trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1]).squeeze()
    
    print('test shape:', preds.shape, trues.shape)

    return preds, trues

In [35]:
for model_config, model in models:
    
    test_data, test_loader = data_provider(model_config, 'test')

    y_pred, y_true = pytorch_test_forecasting(model, model_config, test_data, test_loader)
    
    mae = mean_absolute_error(y_true, y_pred)
    
    print("Test MAE:", mae)
    
    if len(y_pred.shape) == 1:
        df = pd.DataFrame(np.stack((y_pred, y_true)).T, columns = ['pred', 'y'])
        
        fig = px.line(df)
        
        fig.update_layout(title = f'Test Forecasting for {dt_name} with ' + model_config.name )
        fig.show()
    else:
        print(">>> No 24h forecasting visualization")

>>> test 2631
test shape: (41, 64, 1, 1) (41, 64, 1, 1)
test shape: (2624,) (2624,)
Test MAE: 0.15687266


>>> test 2608
test shape: (40, 64, 24, 1) (40, 64, 24, 1)
test shape: (2560, 24) (2560, 24)
Test MAE: 0.6230975
>>> No 24h forecasting visualization


### Forecasting All Horizon of Future Energy Demand Dataset (test.csv) by Using Predictions Instead of Target Variables Observed in Time
In short: Recursive Forecasting

In [36]:
def pytorch_recursive_forecasting(model, data_set, seq_len, pred_len):
    model.eval()

    with torch.no_grad():
        
        for i, data in enumerate(tqdm(data_set)):
            
            if i == len(data_set): # required due to a bug which is infinite iteration of data_set
                break  
            
            if i % pred_len != 0:
                continue

            seq_x, seq_y, seq_x_mark, seq_y_mark = data 
            seq_x, seq_y, seq_x_mark, seq_y_mark = torch.tensor([seq_x]), torch.tensor([seq_y]), \
                                                    torch.tensor([seq_x_mark]), torch.tensor([seq_y_mark])

            if i > 0:
                b = max(0, i-seq_len)
                seq_x[0, -i:,-1] = pred_all[b:i].flatten().clone()
                #seq_y[0, :,-1] = pred_all[i-pred_len:i+pred_len].flatten().clone()
                seq_y[0, :pred_len,-1] = pred_all[i-1:i+pred_len].flatten().clone()
            
            x = seq_x.float().to(device)
            y = seq_y.float()

            x_mark = seq_x_mark.float().to(device)
            y_mark = seq_y_mark.float().to(device)

            dec_inp = torch.zeros_like(y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)

            if args.output_attention:
                outputs = model(x, x_mark, dec_inp, y_mark)[0]
            else:
                outputs = model(x, x_mark, dec_inp, y_mark)

            f_dim = -1 if args.features == 'MS' else 0
            y = outputs[:, -args.pred_len:, f_dim:]
            y = y[:, -args.pred_len:, f_dim:].to(device)

            pred = outputs.detach().cpu()
            y = y.detach().cpu()
        
            pred_all = torch.cat([pred_all, pred[0][:][:,-1][:pred_len] ]) if i>0 else pred[0][:][:,-1][:pred_len]
            #y_all = torch.cat([y_all, y]) if i > 0 else y

    pred_all = np.array(pred_all)
    y_true = data_set.data_y[seq_len:,-1]
    if len(pred_all) < len(y_true):
        y_true = y_true[:len(pred_all)]
    return pred_all, y_true


In [ ]:
for model_config, model in models:
    
    data_set, data_loader = data_provider(model_config, 'test')

    # model_24h can also be used for 1h hour forecasting
    # data_set.label_len = 1
    # data_set.pred_len = 1
    # pred_len = 1

    seq_len = model_config.seq_len
    pred_len = model_config.pred_len

    pred_all, y_true = pytorch_recursive_forecasting(model, data_set, seq_len, pred_len)

    df = pd.DataFrame(np.stack((pred_all, y_true)).T, columns = ['pred', 'y'])
    fig = px.line(df)
    fig.update_layout(title = 'Test Forecasting with ' + model_config.name )
    fig.show()

<div class="alert alert-block alert-info">
⚠️ iTransformer models fail with the recursive forecasting!!! So, they are not usable and should be probed later.
</div>